In [ ]:
def _lr(cb): return cb.pg['lr']
def _beta1(cb): return cb.pg['betas'][0]
record = RecorderCB(lr = _lr, mom = _beta1)
lr = 2e-2
epochs = 30

# load data

tmax = epochs * len(dls.train)


In [ ]:
import wandb

# Define the baseline sweep configuration
sweep_config = {
    'name': 'Dynamic Sweep Configuration',
    'method': 'grid',  # This example uses grid search; adjust as needed.
    'metric': {
        'name': 'validation_loss',
        'goal': 'minimize'   
    },
    'parameters': {
        'data_normalization': {
            'values': [False, True]
        },
        'batch_norm': {
            'values': [None, 'nn.BatchNorm2d']  # Placeholder; actual implementation may vary.
        },
        'activation_function': {
            'values': ['relu', {'leaky_relu': 0.1}] 
        },
        'lr_schedule': {
            'values': [None, 'cosine_annealing']
        },
        'optimizer': {
            'values': ['optim.sgd', 'optim.AdamW']
        },
        'data_augmentation': {
            'values': [False, True]
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project="your_project_name")

In [ ]:
if sweep_config['parameters']['lr_schedule']: sched = partial(lr_scheduler.OneCycleLR, max_lr=lr, total_steps=tmax) else: sched = None

# nn.ReLU else, i might just edit callback to do this
astats = ActivationStats(fc.risinstance(GeneralRelu))

# if leaky_relu
if leaky:
    iw = partial(init_weights, leaky=0.1)
    act_gr = partial(GeneralRelu, leak=0.1, sub=0.4)
else:
    iw = init_weights
    act_gr = GeneralRelu

model = get_model(act_gr,norm=nn.BatchNorm2d).apply(iw)

xtra = [BatchSchedCB(sched),record]
cbs = [DeviceCB(), ProgressCB(plot=True), MetricsCB(), astats]
learn = TrainLearner(model, dls, F.cross_entropy, lr=lr, cbs=cbs+xtra, opt_func=optim.AdamW)

In [3]:
import wandb

def create_sweep_config(step):
    """Generates sweep configurations with names based on active parameters."""
    base_config = {
        'method': 'grid',
        'metric': {'name': 'validation_loss', 'goal': 'minimize'},
        'parameters': {
            'epochs': {'value': 50}
        }
    }

    # Define parameters for each step
    steps_parameters = [
        {},  # Step 1: Baseline, no additional parameters
        {'data_normalization': True},  # Step 2
        {'batch_norm': 'nn.BatchNorm2d'},  # Step 3
        {'activation_function': 'leaky_relu', 'leak_rate': 0.1},  # Step 4
        {'lr_schedule': 'cosine_annealing'},  # Step 5
        {'optimizer': 'optim.AdamW'},  # Step 6
        {'data_augmentation': True}  # Step 7
    ]
    
    # Update the base configuration with parameters for the current step
    for i, params in enumerate(steps_parameters[:step], start=1):
        base_config['parameters'].update(params)
    
    # Generate a unique name based on active parameters
    active_params = [key for key in steps_parameters[step-1] if steps_parameters[step-1][key]]
    unique_name = "Step {} - {}".format(step, ", ".join(active_params))
    if not active_params:  # For the baseline model
        unique_name = "Step 1 - Baseline Model"
    
    base_config['name'] = unique_name
    
    return base_config

# Example usage: Generate sweep configurations for the first 4 steps
for step in range(1, 8):
    sweep_config = create_sweep_config(step)
    #sweep_id = wandb.sweep(sweep_config, project="your_project_name")
    print(f"Sweep ID for '{sweep_config['name']}': ") #{sweep_id}")

Sweep ID for 'Step 1 - Baseline Model': 
Sweep ID for 'Step 2 - data_normalization': 
Sweep ID for 'Step 3 - batch_norm': 
Sweep ID for 'Step 4 - activation_function, leak_rate': 
Sweep ID for 'Step 5 - lr_schedule': 
Sweep ID for 'Step 6 - optimizer': 
Sweep ID for 'Step 7 - data_augmentation': 


In [ ]:
tmax = epochs * len(dls.train)

if sweep_config['parameters']['lr_schedule']: sched = partial(lr_scheduler.OneCycleLR, max_lr=lr, total_steps=tmax) else: sched = None

# this is fine need to check if i can do this with wandb
astats = ActivationStats(fc.risinstance(GeneralRelu))

# if leaky_relu
if leaky:
    iw = partial(init_weights, leaky=0.1)
    act_gr = partial(GeneralRelu, leak=0.1, sub=0.4)
elif init:
    iw = init_weights
    act_gr = GeneralRelu
else:
    iw = None
    act_gr = GeneralRelu

if norm:
    norm = nn.BatchNorm2d


model = get_model(act_gr,norm=norm).apply(iw) # need to check if i can apply None

xtra = [BatchSchedCB(sched),record] # what happens if BatchSChedCB is None
cbs = [DeviceCB(), ProgressCB(plot=True), MetricsCB(), astats] # Nothing

# just aug
learn = TrainLearner(model, dls, F.cross_entropy, lr=lr, cbs=cbs+xtra, opt_func=sweep_config['parameters']['optimizer'])